# Optimus & Cell Ranger Data Comparisons

In This notebook, we compare optimus and cell ranger outputs. I initially examined the count matrices but realized there were large discrepancies, so am now looking at the bam files. The current summary is that we should have outputs that are equivalent to cell ranger from the alignment stage, and I am investigating whether there are discrepancies in the way that we're tagging the bam files. 

In [6]:
import sctools
import numpy as np
import collections
import scipy.sparse as sp_sparse
import tables
import pysam
from collections import Counter

The following cells download and load the Optimus and cell ranger count matrices

In [17]:
%%bash
gsutil cp gs://broad-dsde-mint-dev-cromwell-execution/cromwell-executions/TestOptimusScientific/fd674a6d-373f-4d3f-9d17-ed9b4de58535/call-target/Optimus/36b08252-3b0d-4998-bf99-05d779e5599f/call-MergeCountFiles/sparse_counts.npz ./optimus/sparse_counts.npz
gsutil cp gs://broad-dsde-mint-dev-cromwell-execution/cromwell-executions/TestOptimusScientific/fd674a6d-373f-4d3f-9d17-ed9b4de58535/call-target/Optimus/36b08252-3b0d-4998-bf99-05d779e5599f/call-MergeCountFiles/sparse_counts_col_index.npy ./optimus/sparse_counts_col_index.npy
gsutil cp gs://broad-dsde-mint-dev-cromwell-execution/cromwell-executions/TestOptimusScientific/fd674a6d-373f-4d3f-9d17-ed9b4de58535/call-target/Optimus/36b08252-3b0d-4998-bf99-05d779e5599f/call-MergeCountFiles/sparse_counts_row_index.npy ./optimus/sparse_counts_row_index.npy

gsutil cp gs://broad-dsde-mint-dev-cromwell-execution/cromwell-executions/cellranger/92e66952-18b3-4f16-a95d-2d6eb15c2081/call-CellRanger/synthetic/outs/raw_gene_bc_matrices_h5.h5 ./cell_ranger/raw_gene_bc_matrices_h5.h5

Copying gs://broad-dsde-mint-dev-cromwell-execution/cromwell-executions/TestOptimusScientific/fd674a6d-373f-4d3f-9d17-ed9b4de58535/call-target/Optimus/36b08252-3b0d-4998-bf99-05d779e5599f/call-MergeCountFiles/sparse_counts.npz...
/ [1 files][  2.0 KiB/  2.0 KiB]                                                
Operation completed over 1 objects/2.0 KiB.                                      
Copying gs://broad-dsde-mint-dev-cromwell-execution/cromwell-executions/TestOptimusScientific/fd674a6d-373f-4d3f-9d17-ed9b4de58535/call-target/Optimus/36b08252-3b0d-4998-bf99-05d779e5599f/call-MergeCountFiles/sparse_counts_col_index.npy...
/ [1 files][  2.4 MiB/  2.4 MiB]                                                
Operation completed over 1 objects/2.4 MiB.                                      
Copying gs://broad-dsde-mint-dev-cromwell-execution/cromwell-executions/TestOptimusScientific/fd674a6d-373f-4d3f-9d17-ed9b4de58535/call-target/Optimus/36b08252-3b0d-4998-bf99-05d779e5599f/call-MergeCountF

In [20]:
optimus_matrix = 'optimus/sparse_counts'
op_cm = sctools.count.CountMatrix.load(optimus_matrix)

GeneBCMatrix = collections.namedtuple('GeneBCMatrix', ['gene_ids', 'gene_names', 'barcodes', 'matrix'])
 
def get_matrix_from_h5(filename, genome):
    with tables.open_file(filename, 'r') as f:
        try:
            group = f.get_node(f.root, genome)
        except tables.NoSuchNodeError:
            print("That genome does not exist in this file.")
            return None
        gene_ids = getattr(group, 'genes').read()
        gene_names = getattr(group, 'gene_names').read()
        barcodes = getattr(group, 'barcodes').read()
        data = getattr(group, 'data').read()
        indices = getattr(group, 'indices').read()
        indptr = getattr(group, 'indptr').read()
        shape = getattr(group, 'shape').read()
        matrix = sp_sparse.csc_matrix((data, indices, indptr), shape=shape)
        return GeneBCMatrix(gene_ids, gene_names, barcodes, matrix)
 
filtered_matrix_h5 = "cell_ranger/raw_gene_bc_matrices_h5.h5"
genome = "GRCh38"
cr_cm = get_matrix_from_h5(filtered_matrix_h5, genome)

In [22]:
print(f'cell ranger total counts: {cr_cm.matrix.sum()}')
print(f'optimus total counts: {op_cm.matrix.sum()}')

cell ranger total counts: 246700
optimus total counts: 734


# Bam Comparisons

We have an obvious problem somewhere before the count matrix. Examine the two .bam files for tag disparities. If they exist, that will tell us which method is causing the problem by isolating the offending tag. If there are _not_ tag disparities, it tells us a tag consumer is the problem. 

We have a bit of data going into this: we exchanged the count matrix construction method and the problem persisted, which suggests that an upstream module is likely the problem. 

In [3]:
# identify the two bam files
op_bam = 'optimus_out/merged.bam'
cr_bam = 'cellranger_out/cellranger_results/possorted_genome_bam.bam'

In [10]:
op = pysam.AlignmentFile(op_bam)
cr = pysam.AlignmentFile(cr_bam)

These bams should have alignments to chromosome 1 and 2

In [24]:
cr_chr1_records = sum(1 for _ in cr.fetch(contig='1'))
op_chr1_records = sum(1 for _ in op.fetch(contig='1'))
print(f'Optimus Chromosome 1 Records: {op_chr1_records}')
print(f'Cellranger Chromosome 1 Records: {cr_chr1_records}')

Optimus Chromosome 1 Records: 11833
Cellranger Chromosome 1 Records: 1170508


In [23]:
cr_chr2_records = sum(1 for _ in cr.fetch(contig='2'))
op_chr2_records = sum(1 for _ in op.fetch(contig='2'))
print(f'Optimus Chromosome 2 Records: {op_chr2_records}')
print(f'Cellranger Chromosome 2 Records: {cr_chr2_records}')

Optimus Chromosome 2 Records: 0
Cellranger Chromosome 2 Records: 134053


In [26]:
cr_unmapped_records = sum(1 for record in cr if record.is_unmapped)
op_unmapped_records = sum(1 for record in op if record.is_unmapped)
print(f'Optimus unmapped records: {op_unmapped_records}')
print(f'Cellranger unmapped records: {cr_unmapped_records}')

Optimus unmapped records: 1122451
Cellranger unmapped records: 134321


This suggests the problem is alignment. Find the parameters that both methods are using. 

Optimus:
```bash
    STAR \
      --runMode alignReads \
      --runThreadN ${cpu} \
      --genomeDir genome_reference \
      --readFilesIn ${bam_input} \
      --outSAMtype BAM Unsorted \
      --outSAMattributes All \
      --outFilterMultimapNmax 1 \
      --outSAMunmapped Within \
      --outSAMprimaryFlag AllBestScore \
      --readFilesType SAM SE \
      --readFilesCommand samtools view -h \
      --runRNGseed 777
```

Cellranger:
```python
    'STAR', 
    '--genomeDir', self.reference_star_path,          # we use the same reference
    '--readFilesIn', read1_fastq_fn, read2_fastq_fn,  # fastq inputs
    '--outSAMmultNmax', -1,                           # default
    '--runThreadN', str(threads),
    '--readNameSeparator', 'space',
    '--outSAMunmapped', 'Within',
    '--outSAMtype', 'SAM',
    '--outStd', 'SAM',
    '--outSAMorder', 'PairedKeepInputOrder',          # for use with runThreadN > 1
    ```

Proposed matching arguments: 
```bash
    STAR \
      --runMode alignReads \
      --runThreadN ${cpu} \  
      --genomeDir genome_reference \
      --readFilesIn ${bam_input} \
      --outSAMtype BAM Unsorted \
      --outSAMattributes All \
      --outSAMunmapped Within \
      --readFilesType SAM SE \
      --readFilesCommand samtools view -h \
      --runRNGseed 777
```


At this point I reran optimus with updated parameters

In [4]:
%%bash
gsutil cp gs://broad-dsde-mint-dev-cromwell-execution/cromwell-executions/cellranger/92e66952-18b3-4f16-a95d-2d6eb15c2081/call-CellRanger/synthetic/outs/possorted_genome_bam.bam ./cell_ranger/possorted_genome_bam.bam
gsutil cp gs://broad-dsde-mint-dev-cromwell-execution/cromwell-executions/cellranger/92e66952-18b3-4f16-a95d-2d6eb15c2081/call-CellRanger/synthetic/outs/possorted_genome_bam.bam.bai ./cell_ranger/possorted_genome_bam.bam.bai
gsutil cp gs://broad-dsde-mint-dev-cromwell-execution/cromwell-executions/TestOptimusScientific/fd674a6d-373f-4d3f-9d17-ed9b4de58535/call-target/Optimus/36b08252-3b0d-4998-bf99-05d779e5599f/call-MergeSorted/merged.bam ./optimus/merged.bam
samtools index optimus/merged.bam

Copying gs://broad-dsde-mint-dev-cromwell-execution/cromwell-executions/cellranger/92e66952-18b3-4f16-a95d-2d6eb15c2081/call-CellRanger/synthetic/outs/possorted_genome_bam.bam...
\ [1 files][188.9 MiB/188.9 MiB]  640.1 KiB/s                                   
Operation completed over 1 objects/188.9 MiB.                                    
Copying gs://broad-dsde-mint-dev-cromwell-execution/cromwell-executions/cellranger/92e66952-18b3-4f16-a95d-2d6eb15c2081/call-CellRanger/synthetic/outs/possorted_genome_bam.bam.bai...
- [1 files][  1.6 MiB/  1.6 MiB]                                                
Operation completed over 1 objects/1.6 MiB.                                      
Copying gs://broad-dsde-mint-dev-cromwell-execution/cromwell-executions/TestOptimusScientific/fd674a6d-373f-4d3f-9d17-ed9b4de58535/call-target/Optimus/36b08252-3b0d-4998-bf99-05d779e5599f/call-MergeSorted/merged.bam...
\ [1 files][202.0 MiB/202.0 MiB]    2.8 MiB/s                                   
Operation co

In [48]:
op_bam = 'optimus/merged.bam'
op = pysam.AlignmentFile(op_bam)

cr_bam = 'cell_ranger/possorted_genome_bam.bam'
cr = pysam.AlignmentFile(cr_bam)

In [13]:
print(sum(1 for _ in op))
print(sum(1 for _ in cr))

3055752
3055752


In [15]:
op_chr1_records = sum(1 for _ in op.fetch(contig='1'))
print(f'Optimus Chromosome 1 Records: {op_chr1_records}')

cr_chr1_records = sum(1 for _ in cr.fetch(contig='1'))
print(f'Cell Ranger Chromosome 1 Records: {cr_chr1_records}')

Optimus Chromosome 1 Records: 1170508
Cell Ranger Chromosome 1 Records: 1170508


In [16]:
op_chr2_records = sum(1 for _ in op.fetch(contig='2'))
print(f'Optimus Chromosome 1 Records: {op_chr2_records}')

cr_chr2_records = sum(1 for _ in cr.fetch(contig='2'))
print(f'Cell Ranger Chromosome 1 Records: {cr_chr2_records}')

Optimus Chromosome 1 Records: 134053
Cell Ranger Chromosome 1 Records: 134053


# Compare tag abundances

In [49]:
# pile up number of times each tag is seen
cr_counter = Counter()
for record in cr:
    for (k, v) in record.get_tags():
        cr_counter[k] += 1

In [50]:
# pile up number of times each tag is seen
op_counter = Counter()
for record in op:
    for (k, v) in record.get_tags():
        op_counter[k] += 1

In [51]:
cr_counter

Counter({'NH': 3055752,
         'HI': 3055752,
         'AS': 3055752,
         'nM': 3055752,
         'RE': 2921431,
         'CR': 3055752,
         'CY': 3055752,
         'CB': 3008830,
         'UR': 3055752,
         'UY': 3055752,
         'UB': 3055024,
         'BC': 3055752,
         'QT': 3055752,
         'RG': 3055752,
         'MM': 1573766,
         'TX': 755521,
         'GX': 755521,
         'GN': 755521,
         'uT': 134321})

In [ ]:
op_bam = 'optimus/merged.bam'
op = pysam.AlignmentFile(op_bam)

cr_bam = 'cell_ranger/possorted_genome_bam.bam'
cr = pysam.AlignmentFile(cr_bam)

In [54]:
print('keys in cell ranger and not in optimus')
set(cr_counter.keys()).difference(op_counter.keys())

keys in cell ranger and not in optimus


{'BC', 'GN', 'GX', 'MM', 'QT', 'RE', 'TX'}

In [55]:
print('keys in optimus and not in cell ranger')
set(op_counter.keys()).difference(cr_counter.keys())

keys in optimus and not in cell ranger


{'GE', 'GS', 'MD', 'NM', 'XF', 'jI', 'jM'}

In [56]:
print('difference in key abundance for keys in both pipelines')
intersection = set(op_counter.keys()).intersection(cr_counter.keys())
for k in intersection:
    print(f'{k}, optimus - cell ranger: {op_counter[k] - cr_counter[k]}')

difference in key abundance for keys in both pipelines
nM, optimus - cell ranger: 0
UB, optimus - cell ranger: 728
uT, optimus - cell ranger: 0
CY, optimus - cell ranger: 0
CR, optimus - cell ranger: 0
CB, optimus - cell ranger: 3569
UY, optimus - cell ranger: 0
UR, optimus - cell ranger: 0
RG, optimus - cell ranger: 0
AS, optimus - cell ranger: 0
NH, optimus - cell ranger: 0
HI, optimus - cell ranger: 0


Given the magnitude of difference in the count matrices, intersecting keys cannot be driving the discrepancy we observe, although non-intersecting keys could. 

# Examine differences in flags

In [72]:
op_bam = 'optimus/merged.bam'
op = pysam.AlignmentFile(op_bam)

cr_bam = 'cell_ranger/possorted_genome_bam.bam'
cr = pysam.AlignmentFile(cr_bam)

In [75]:
cr_flag_counter = Counter()
for record in cr:
    for i, val in enumerate(bin(record.flag)[2:]):
        if val == '1':
            cr_flag_counter[i] += 1

In [76]:
op_flag_counter = Counter()
for record in op:
    for i, val in enumerate(bin(record.flag)[2:]):
        if val == '1':
            op_flag_counter[i] += 1

In [79]:
op_flag_counter

Counter({0: 2474789, 4: 578686, 6: 68129})

In [80]:
cr_flag_counter

Counter({0: 2675400, 4: 573725, 6: 23009})

In [81]:
print('difference in flag abundance for keys in both pipelines')
flag_intersection = set(op_flag_counter.keys()).intersection(cr_flag_counter.keys())
for k in flag_intersection:
    print(f'{k}, optimus - cell ranger: {op_flag_counter[k] - cr_flag_counter[k]}')

difference in flag abundance for keys in both pipelines
0, optimus - cell ranger: -200611
4, optimus - cell ranger: 4961
6, optimus - cell ranger: 45120


This might be the culprit... we've got a lot fewer reads with `0` flags than CR does. Because of the way python indexes, this means the second bit (`x4`), which is "reverse complemented". Conversely, optimus sets the 5th and 7th bits more frequently. 